In [1]:
from zmq.devices import ProcessProxy, MonitoredQueue
from multiprocessing import Process
import zmq, time, random 


In [2]:
to_client = 6666
to_server = 6667
to_monitor = 6668
number_of_clients = 3

In [3]:
def server(backend_port):
    print("Connecting a server to queue device")
    context = zmq.Context()
    socket = context.socket(zmq.REP)
    socket.connect("tcp://127.0.0.1:%s" % backend_port)
    server_id = random.randrange(1,10005)
    while True:
        print ("while True") 
        message = socket.recv()
        print ("Received request: ", message ) 
        socket.send_string("Response from %s" % server_id)

In [4]:
def client(frontend_port, client_id):
    print ("Connecting a worker #%s to queue device" % client_id)
    context = zmq.Context()
    socket = context.socket(zmq.REQ)
    socket.connect("tcp://127.0.0.1:%s" % frontend_port)
    #  Do 10 requests, waiting each time for a response
    for request in range (1,5):
        print("Sending request #%s" % request)
        socket.send_string ("Request fron client: %s" % client_id)
        #  Get the reply.
        message = socket.recv()
        print("Received reply ", request, "[", message, "]")

In [5]:
def monitor(monitor_port):
    print("Starting monitoring process")
    context = zmq.Context()
    socket = context.socket(zmq.SUB)
    print("Collecting updates from server...")
    socket.connect ("tcp://127.0.0.1:%s" % monitor_port)
    #socket.setsockopt(zmq.SUBSCRIBE, "")
    while True:
        string = socket.recv()
        print ("Monitoring Client: %s" % string)


In [6]:
def monitor_device(frontend_port, backend_port, monitor_port):
    in_prefix  = b'in'
    out_prefix = b'out'
    device = MonitoredQueue(zmq.DEALER, zmq.ROUTER, zmq.PUB)
    #queuedevice = ProcessProxy(zmq.DEALER, zmq.ROUTER)
    device.bind_in('tcp://localhost:{}'.format(frontend_port))
    device.bind_out('tcp://localhost:{}'.format(backend_port))
    device.bind_mon('tcp://localhost:{}'.format(monitor_port))
#queuedevice.setsockopt_in(zmq.RCVHWM, 4)
#queuedevice.setsockopt_out(zmq.SNDHWM, 1)
#queuedevice.bind_in('tcp://*:{}'.format(to_client))
#queuedevice.connect_out('tcp://localhost:{}'.format(to_server))
#queuedevice.setsockopt_in(zmq.IDENTITY, b'ROUTER')
#queuedevice.setsockopt_out(zmq.IDENTITY, b'DEALER')
    device.setsockopt_in(zmq.RCVHWM, 1)
    device.setsockopt_out(zmq.SNDHWM, 1)
    device.start()

In [10]:
p_device = Process(target=monitor_device, args=(to_client, to_server, to_monitor))
p_device.start()

p_server = Process(target=server, args=(to_server,))
p_server.start() 

p_monitor = Process(target=monitor, args=(to_monitor,))
p_monitor.start() 

time.sleep(2)
for client_id in range(number_of_clients):
    Process(target=client, args=(to_client, client_id,)).start()
    
time.sleep(10)
p_server.terminate()
p_monitor.terminate()
p_device.terminate()

Process Process-11:


Connecting a server to queue device


Traceback (most recent call last):


while True


  File "/Users/csaintje/bin/anaconda3/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/Users/csaintje/bin/anaconda3/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)


Starting monitoring process


  File "<ipython-input-6-35165c3dd07c>", line 17, in monitor_device
    device.start()


  File "/Users/csaintje/bin/anaconda3/lib/python3.5/site-packages/zmq/devices/basedevice.py", line 182, in start
    return self.run()
  File "/Users/csaintje/bin/anaconda3/lib/python3.5/site-packages/zmq/devices/basedevice.py", line 170, in run
    self.run_device()
  File "/Users/csaintje/bin/anaconda3/lib/python3.5/site-packages/zmq/devices/monitoredqueuedevice.py", line 26, in run_device
    ins,outs,mons = self._setup_sockets()
  File "/Users/csaintje/bin/anaconda3/lib/python3.5/site-packages/zmq/devices/proxydevice.py", line 43, in _setup_sockets
    ins,outs = Device._setup_sockets(self)
  File "/Users/csaintje/bin/anaconda3/lib/python3.5/site-packages/zmq/devices/basedevice.py", line 148, in _setup_sockets
    ins.bind(iface)
  File "zmq/backend/cython/socket.pyx", line 495, in zmq.backend.cython.socket.Socket.bind (zmq/backend/cython/socket.c:5653)
  File "zmq/backend/cython/checkrc.pxd", line 25, in zmq.backend.cython.checkrc._check_rc (zmq/backend/cython/socket.c:10014)
zmq.

Connecting a worker #0 to queue device
Sending request #1
Connecting a worker #1 to queue device
Sending request #1
Connecting a worker #2 to queue device
Sending request #1
Received reply  1 [ b'Response from 5223' ]
Sending request #2
Received reply  2 [ b'Response from 5223' ]
Sending request #3
Received reply  1 [ b'Response from 5223' ]
Sending request #2
Received reply  3 [ b'Response from 5223' ]
Sending request #4
Received reply  2 [ b'Response from 5223' ]
Sending request #3
Received reply  4 [ b'Response from 5223' ]
Received reply  3 [ b'Response from 5223' ]
Sending request #4
Received reply  4 [ b'Response from 5223' ]
Received reply  1 [ b'Response from 5223' ]
Sending request #2
Received reply  2 [ b'Response from 5223' ]
Sending request #3
Received reply  3 [ b'Response from 5223' ]
Sending request #4
Received reply  4 [ b'Response from 5223' ]
